<a href="https://colab.research.google.com/github/alan713/alan1/blob/master/DL_hw2_1_1Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
# This will open a file browser
uploaded = files.upload()

Saving tiny-imagenet-200.zip to tiny-imagenet-200.zip


In [3]:
print("\n📦 Installing required libraries...")
!pip install -q torch torchvision pillow numpy
print("✅ All libraries installed successfully!")

# Import libraries
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


📦 Installing required libraries...
✅ All libraries installed successfully!
PyTorch version: 2.8.0+cu126
CUDA available: False


In [4]:
import zipfile
import os
from pathlib import Path

# Path to your uploaded zip file (in Colab's local storage)
ZIP_PATH = '/content/tiny-imagenet-200.zip'  # Default location for uploaded files
EXTRACT_TO = '/content/'  # Extract to Colab's local storage

print("="*70)
print("📦 UNZIPPING TINY IMAGENET")
print("="*70)

# First, let's find the zip file
print("Searching for zip file...")
zip_files = [f for f in os.listdir('/content/') if f.endswith('.zip')]
print(f"Found {len(zip_files)} zip file(s): {zip_files}")

if zip_files:
    # Use the first zip file found (or the one matching our name)
    if 'tiny-imagenet-200.zip' in zip_files:
        ZIP_PATH = '/content/tiny-imagenet-200.zip'
    else:
        ZIP_PATH = f'/content/{zip_files[0]}'
        print(f"⚠️ Using: {zip_files[0]}")

if os.path.exists(ZIP_PATH):
    print(f"✅ Found zip file: {ZIP_PATH}")

    # Get file size
    file_size_mb = os.path.getsize(ZIP_PATH) / (1024 * 1024)
    print(f"📊 File size: {file_size_mb:.2f} MB")

    print("⏳ Unzipping... (this takes ~2-3 minutes)")

    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_TO)

    print("✅ Unzipping complete!")

    # Find the extracted folder
    DATASET_PATH = '/content/tiny-imagenet-200'

    if os.path.exists(DATASET_PATH):
        print(f"\n📂 Dataset location: {DATASET_PATH}")

        # Explore structure
        print("\n📊 Dataset Structure:")
        for item in os.listdir(DATASET_PATH):
            item_path = os.path.join(DATASET_PATH, item)
            if os.path.isdir(item_path):
                num_items = len(os.listdir(item_path))
                print(f"  📁 {item}/ ({num_items} items)")
            else:
                print(f"  📄 {item}")

        # Check train folder
        train_path = os.path.join(DATASET_PATH, 'train')
        if os.path.exists(train_path):
            classes = [d for d in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, d))]
            print(f"\n✅ Training set: {len(classes)} classes")

            # Sample a class to see structure
            if classes:
                sample_class = classes[0]
                sample_path = os.path.join(train_path, sample_class, 'images')
                if os.path.exists(sample_path):
                    num_images = len([f for f in os.listdir(sample_path) if f.endswith('.JPEG')])
                    print(f"✅ Sample class '{sample_class}': {num_images} images")
    else:
        print(f"❌ Could not find extracted folder at: {DATASET_PATH}")

else:
    print(f"❌ ZIP file not found!")
    print(f"\nSearched at: {ZIP_PATH}")
    print("\n💡 To upload the file:")
    print("1. Click the 📁 folder icon on the left sidebar")
    print("2. Click the ⬆️ upload button")
    print("3. Select your tiny-imagenet-200.zip file")
    print("4. Wait for upload to complete")
    print("5. Re-run this cell")

📦 UNZIPPING TINY IMAGENET
Searching for zip file...
Found 1 zip file(s): ['tiny-imagenet-200.zip']
✅ Found zip file: /content/tiny-imagenet-200.zip
📊 File size: 236.61 MB
⏳ Unzipping... (this takes ~2-3 minutes)
✅ Unzipping complete!

📂 Dataset location: /content/tiny-imagenet-200

📊 Dataset Structure:
  📁 train/ (200 items)
  📄 words.txt
  📁 test/ (1 items)
  📄 wnids.txt
  📁 val/ (2 items)

✅ Training set: 200 classes
✅ Sample class 'n04118538': 500 images


In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import shutil
from pathlib import Path
import random
from typing import Tuple, List, Dict
import json

class TinyImageNetSubsetCreator:
    """
    Creates a custom subset of Tiny ImageNet with specified number of classes.
    Prepares data according to AlexNet specifications.
    """

    def __init__(self,
                 source_path: str,
                 output_path: str,
                 num_classes: int = 100,
                 train_size: int = 30000,
                 val_size: int = 10000,
                 test_size: int = 10000):
        """
        Initialize dataset creator for Tiny ImageNet.

        Args:
            source_path: Path to extracted tiny-imagenet-200 folder
            output_path: Path where processed dataset will be saved
            num_classes: Number of classes to include (default: 100)
            train_size: Total training images (default: 30000)
            val_size: Total validation images (default: 10000)
            test_size: Total testing images (default: 10000)
        """
        self.source_path = Path(source_path)
        self.output_path = Path(output_path)
        self.num_classes = num_classes
        self.train_size = train_size
        self.val_size = val_size
        self.test_size = test_size

        # Calculate per-class splits
        self.train_per_class = train_size // num_classes  # 300
        self.val_per_class = val_size // num_classes      # 100
        self.test_per_class = test_size // num_classes    # 100

        # Tiny ImageNet has 500 images per class in train
        # We need 500 total per class (300 train + 100 val + 100 test)
        self.images_per_class = self.train_per_class + self.val_per_class + self.test_per_class

        # Create output directories
        self.splits = ['train', 'val', 'test']
        self._create_directories()

    def _create_directories(self):
        """Create directory structure for train/val/test splits."""
        for split in self.splits:
            split_path = self.output_path / split
            split_path.mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {split_path}")

    def select_classes(self) -> List[str]:
        """
        Randomly select specified number of classes from Tiny ImageNet.

        Returns:
            List of selected class directory names (WordNet IDs)
        """
        # Get all class directories from train folder
        train_path = self.source_path / 'train'
        all_classes = [d.name for d in train_path.iterdir()
                      if d.is_dir() and not d.name.startswith('.')]

        # Randomly select num_classes
        random.seed(42)  # For reproducibility
        selected_classes = random.sample(all_classes, self.num_classes)

        print(f"Selected {self.num_classes} classes from {len(all_classes)} available classes")
        return selected_classes

    def collect_images_per_class(self, class_name: str) -> List[Path]:
        """
        Collect images for a given class from Tiny ImageNet.

        Args:
            class_name: Name of the class directory (WordNet ID)

        Returns:
            List of image file paths
        """
        # In Tiny ImageNet, images are in train/class_name/images/
        class_images_path = self.source_path / 'train' / class_name / 'images'

        # Get all JPEG images
        all_images = list(class_images_path.glob('*.JPEG'))

        # Tiny ImageNet has 500 images per class
        if len(all_images) < self.images_per_class:
            print(f"Warning: Class {class_name} has only {len(all_images)} images")
            return all_images

        # Take exactly the number we need
        selected_images = random.sample(all_images, self.images_per_class)
        return selected_images

    def split_and_copy_images(self, images: List[Path], class_idx: int, class_name: str):
        """
        Split images into train/val/test sets and copy to output directories.

        Args:
            images: List of image paths for a class
            class_idx: Integer index for the class (0-99)
            class_name: Name of the class (WordNet ID)
        """
        # Shuffle images for random split
        random.shuffle(images)

        # Split images according to specified sizes
        train_images = images[:self.train_per_class]
        val_images = images[self.train_per_class:self.train_per_class + self.val_per_class]
        test_images = images[self.train_per_class + self.val_per_class:
                            self.train_per_class + self.val_per_class + self.test_per_class]

        # Create class subdirectories and copy images
        splits_data = {
            'train': train_images,
            'val': val_images,
            'test': test_images
        }

        for split, split_images in splits_data.items():
            # Create class directory within split
            class_dir = self.output_path / split / f"class_{class_idx:03d}_{class_name}"
            class_dir.mkdir(exist_ok=True)

            # Copy images to class directory
            for img_path in split_images:
                dest_path = class_dir / img_path.name
                shutil.copy2(img_path, dest_path)

        print(f"Class {class_idx:3d} ({class_name}): "
              f"Train={len(train_images)}, Val={len(val_images)}, Test={len(test_images)}")

    def create_dataset(self):
        """
        Main method to create the complete dataset with all splits.
        """
        print("=" * 70)
        print("Starting Tiny ImageNet Subset Creation")
        print("=" * 70)

        # Set random seed for reproducibility
        random.seed(42)

        # Select classes
        selected_classes = self.select_classes()

        # Process each class
        for class_idx, class_name in enumerate(selected_classes):
            print(f"Processing class {class_idx + 1}/{self.num_classes}: ", end="")

            # Collect images for this class
            images = self.collect_images_per_class(class_name)

            # Split and copy images
            self.split_and_copy_images(images, class_idx, class_name)

        # Save class mapping
        class_mapping = {i: name for i, name in enumerate(selected_classes)}
        mapping_file = self.output_path / 'class_mapping.json'
        with open(mapping_file, 'w') as f:
            json.dump(class_mapping, f, indent=2)

        print("\n" + "=" * 70)
        print("Dataset creation completed!")
        print(f"Total images - Train: {self.train_size}, Val: {self.val_size}, Test: {self.test_size}")
        print(f"Class mapping saved to: {mapping_file}")
        print("=" * 70)


class AlexNetImageNetDataset(Dataset):
    """
    Custom Dataset class for loading ImageNet data prepared according to AlexNet specifications.

    Data preprocessing as per AlexNet paper (Section 2):
    - Images resized to 256x256 (rescale shorter side to 256, then crop)
    - Random 224x224 crops for training (center crop for validation/test)
    - Horizontal flips for data augmentation (training only)
    - RGB channel normalization
    """

    def __init__(self,
                 root_dir: str,
                 split: str = 'train',
                 transform: transforms.Compose = None):
        """
        Initialize the dataset.

        Args:
            root_dir: Root directory containing train/val/test folders
            split: One of 'train', 'val', or 'test'
            transform: Optional transform to be applied on images
        """
        self.root_dir = Path(root_dir) / split
        self.split = split
        self.transform = transform

        # Load all image paths and labels
        self.image_paths = []
        self.labels = []
        self._load_dataset()

        # Load class mapping
        mapping_file = Path(root_dir) / 'class_mapping.json'
        with open(mapping_file, 'r') as f:
            self.class_mapping = json.load(f)

    def _load_dataset(self):
        """Load all image paths and corresponding labels."""
        class_dirs = sorted([d for d in self.root_dir.iterdir() if d.is_dir()])

        for class_idx, class_dir in enumerate(class_dirs):
            # Tiny ImageNet uses .JPEG extension
            image_files = list(class_dir.glob('*.JPEG')) + \
                         list(class_dir.glob('*.jpg')) + \
                         list(class_dir.glob('*.jpeg')) + \
                         list(class_dir.glob('*.png'))

            for img_path in image_files:
                self.image_paths.append(img_path)
                self.labels.append(class_idx)

        print(f"Loaded {len(self.image_paths)} images for {self.split} split")

    def __len__(self) -> int:
        """Return the total number of images."""
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        """
        Get an image and its label.

        Args:
            idx: Index of the image

        Returns:
            Tuple of (image_tensor, label)
        """
        # Load image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        # Apply transforms
        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        return image, label


def get_alexnet_transforms(split: str = 'train') -> transforms.Compose:
    """
    Get data transforms according to AlexNet paper specifications (Section 2).

    Training transforms:
    - Resize to 256x256 (Tiny ImageNet is 64x64, so we upscale)
    - Random 224x224 crop
    - Random horizontal flip
    - Convert to tensor
    - Normalize using ImageNet mean and std

    Validation/Test transforms:
    - Resize to 256x256
    - Center crop to 224x224
    - Convert to tensor
    - Normalize using ImageNet mean and std

    Args:
        split: One of 'train', 'val', or 'test'

    Returns:
        Composed transforms
    """
    # ImageNet mean and std for normalization (RGB channels)
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    if split == 'train':
        # Training transforms with data augmentation
        transform = transforms.Compose([
            # Resize to 256x256 (upscale from 64x64)
            transforms.Resize((256, 256)),
            # Random crop to 224x224 (as per AlexNet paper)
            transforms.RandomCrop(224),
            # Random horizontal flip for data augmentation
            transforms.RandomHorizontalFlip(p=0.5),
            # Convert PIL Image to Tensor
            transforms.ToTensor(),
            # Normalize using ImageNet statistics
            transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
        ])
    else:
        # Validation/Test transforms without augmentation
        transform = transforms.Compose([
            # Resize to 256x256
            transforms.Resize((256, 256)),
            # Center crop to 224x224
            transforms.CenterCrop(224),
            # Convert PIL Image to Tensor
            transforms.ToTensor(),
            # Normalize using ImageNet statistics
            transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
        ])

    return transform


def create_dataloaders(data_dir: str,
                       batch_size: int = 128,
                       num_workers: int = 2) -> Dict[str, DataLoader]:
    """
    Create DataLoaders for train, validation, and test sets.

    Args:
        data_dir: Root directory containing the prepared dataset
        batch_size: Batch size for training (AlexNet used 128)
        num_workers: Number of worker processes for data loading

    Returns:
        Dictionary containing DataLoaders for each split
    """
    dataloaders = {}

    for split in ['train', 'val', 'test']:
        # Get appropriate transforms
        transform = get_alexnet_transforms(split)

        # Create dataset
        dataset = AlexNetImageNetDataset(
            root_dir=data_dir,
            split=split,
            transform=transform
        )

        # Create dataloader
        # Shuffle training data, don't shuffle val/test
        shuffle = (split == 'train')

        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            pin_memory=True  # Faster data transfer to GPU
        )

        dataloaders[split] = dataloader
        print(f"{split.capitalize()} DataLoader: "
              f"{len(dataset)} images, {len(dataloader)} batches")

    return dataloaders

print("✅ All classes and functions loaded successfully!")

✅ All classes and functions loaded successfully!


In [6]:
print("\n" + "="*70)
print("🚀 CREATING CUSTOM 100-CLASS SUBSET FROM TINY IMAGENET")
print("="*70)

# Paths
SOURCE_PATH = '/content/tiny-imagenet-200'  # Where the zip was extracted
OUTPUT_PATH = '/content/tiny_imagenet_100'  # Save to Colab (faster processing)
# Note: Data will be lost when Colab disconnects. To save permanently, use:
# OUTPUT_PATH = '/content/drive/MyDrive/tiny_imagenet_100'  # Uncomment to save to Drive

print(f"\n📂 Source: {SOURCE_PATH}")
print(f"💾 Output: {OUTPUT_PATH}")
print(f"\n⚠️ NOTE: Output is in Colab's temporary storage")
print("Data will be deleted when session ends.")
print("To save permanently, mount Drive and update OUTPUT_PATH")

# Check if source exists
if not os.path.exists(SOURCE_PATH):
    print("\n❌ ERROR: Tiny ImageNet not found!")
    print("Make sure you ran CELL 3 to unzip the dataset")
else:
    print("\n✅ Source dataset found!")

    # Create the subset
    creator = TinyImageNetSubsetCreator(
        source_path=SOURCE_PATH,
        output_path=OUTPUT_PATH,
        num_classes=100,        # Select 100 out of 200 classes
        train_size=30000,       # 300 per class
        val_size=10000,         # 100 per class
        test_size=10000         # 100 per class
    )

    # Run dataset creation
    creator.create_dataset()

    print("\n" + "="*70)
    print("✅ DATASET CREATION COMPLETED!")
    print("="*70)


🚀 CREATING CUSTOM 100-CLASS SUBSET FROM TINY IMAGENET

📂 Source: /content/tiny-imagenet-200
💾 Output: /content/tiny_imagenet_100

⚠️ NOTE: Output is in Colab's temporary storage
Data will be deleted when session ends.
To save permanently, mount Drive and update OUTPUT_PATH

✅ Source dataset found!
Created directory: /content/tiny_imagenet_100/train
Created directory: /content/tiny_imagenet_100/val
Created directory: /content/tiny_imagenet_100/test
Starting Tiny ImageNet Subset Creation
Selected 100 classes from 200 available classes
Processing class 1/100: Class   0 (n03255030): Train=300, Val=100, Test=100
Processing class 2/100: Class   1 (n01770393): Train=300, Val=100, Test=100
Processing class 3/100: Class   2 (n02099601): Train=300, Val=100, Test=100
Processing class 4/100: Class   3 (n01945685): Train=300, Val=100, Test=100
Processing class 5/100: Class   4 (n01443537): Train=300, Val=100, Test=100
Processing class 6/100: Class   5 (n02815834): Train=300, Val=100, Test=100
Proc

In [7]:
print("\n" + "="*70)
print("📊 CREATING DATALOADERS WITH ALEXNET PREPROCESSING")
print("="*70)

OUTPUT_PATH = '/content/tiny_imagenet_100'  # Match the path from Cell 5
# If you saved to Drive in Cell 5, use: '/content/drive/MyDrive/tiny_imagenet_100'

# Check if dataset was created
if os.path.exists(OUTPUT_PATH):
    # Create DataLoaders
    dataloaders = create_dataloaders(
        data_dir=OUTPUT_PATH,
        batch_size=128,  # AlexNet batch size
        num_workers=2    # Colab works well with 2 workers
    )

    # Verify the data
    print("\n" + "="*70)
    print("🔍 VERIFYING LOADED DATA")
    print("="*70)

    for split_name, dataloader in dataloaders.items():
        # Get one batch
        images, labels = next(iter(dataloader))

        print(f"\n{split_name.upper()} SET:")
        print(f"  ✓ Batch shape: {images.shape}")
        print(f"  ✓ Labels shape: {labels.shape}")
        print(f"  ✓ Image range: [{images.min():.3f}, {images.max():.3f}]")
        print(f"  ✓ Total batches: {len(dataloader)}")
        print(f"  ✓ Total images: {len(dataloader.dataset)}")
        print(f"  ✓ Classes in batch: {len(torch.unique(labels))}")

    print("\n" + "="*70)
    print("✅ ALL DONE! Dataset ready for AlexNet training!")
    print("="*70)

    # Display summary
    print("\n📋 FINAL SUMMARY:")
    print(f"  • Dataset: Tiny ImageNet (100 classes)")
    print(f"  • Location: {OUTPUT_PATH}")
    print(f"  • Training: 30,000 images (300/class)")
    print(f"  • Validation: 10,000 images (100/class)")
    print(f"  • Testing: 10,000 images (100/class)")
    print(f"  • Image size: 224×224 (upscaled from 64×64)")
    print(f"  • Preprocessing: AlexNet specifications")
    print(f"  • Augmentation: Random crops + flips (train only)")

    # Show how to access in future sessions
    print("\n💡 TO USE IN FUTURE SESSIONS:")
    print("```python")
    print("dataloaders = create_dataloaders(")
    print(f"    data_dir='{OUTPUT_PATH}',")
    print("    batch_size=128")
    print(")")
    print("```")

else:
    print(f"\n❌ ERROR: Dataset not found at {OUTPUT_PATH}")
    print("Please run CELL 5 first!")


📊 CREATING DATALOADERS WITH ALEXNET PREPROCESSING
Loaded 30000 images for train split
Train DataLoader: 30000 images, 235 batches
Loaded 10000 images for val split
Val DataLoader: 10000 images, 79 batches
Loaded 10000 images for test split
Test DataLoader: 10000 images, 79 batches

🔍 VERIFYING LOADED DATA


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



TRAIN SET:
  ✓ Batch shape: torch.Size([128, 3, 224, 224])
  ✓ Labels shape: torch.Size([128])
  ✓ Image range: [-2.118, 2.640]
  ✓ Total batches: 235
  ✓ Total images: 30000
  ✓ Classes in batch: 71

VAL SET:
  ✓ Batch shape: torch.Size([128, 3, 224, 224])
  ✓ Labels shape: torch.Size([128])
  ✓ Image range: [-2.118, 2.640]
  ✓ Total batches: 79
  ✓ Total images: 10000
  ✓ Classes in batch: 2

TEST SET:
  ✓ Batch shape: torch.Size([128, 3, 224, 224])
  ✓ Labels shape: torch.Size([128])
  ✓ Image range: [-2.118, 2.640]
  ✓ Total batches: 79
  ✓ Total images: 10000
  ✓ Classes in batch: 2

✅ ALL DONE! Dataset ready for AlexNet training!

📋 FINAL SUMMARY:
  • Dataset: Tiny ImageNet (100 classes)
  • Location: /content/tiny_imagenet_100
  • Training: 30,000 images (300/class)
  • Validation: 10,000 images (100/class)
  • Testing: 10,000 images (100/class)
  • Image size: 224×224 (upscaled from 64×64)
  • Preprocessing: AlexNet specifications
  • Augmentation: Random crops + flips (train 